In [1]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

# Access the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
unstructured_api_key = os.getenv('UNSTRUCTURED_API_KEY')

## PDFs Preprocessing

In [2]:
from unstructured_client import UnstructuredClient
from unstructured_client.models import shared
from unstructured_client.models.errors import SDKError

from unstructured.partition.html import partition_html
from unstructured.partition.pdf import partition_pdf

from unstructured.staging.base import dict_to_elements
import json
from IPython.display import JSON

In [3]:
s = UnstructuredClient(
    api_key_auth=unstructured_api_key
)

In [4]:
filename = "data/2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf"
with open(filename, "rb") as f:
    files=shared.Files(
        content=f.read(), 
        file_name=filename,
    )

req = shared.PartitionParameters(
    files=files,
    strategy='hi_res',
    pdf_infer_table_structure=True,
    languages=["eng"],
)

In [19]:
try:
    resp = s.general.partition(req)
    print(json.dumps(resp.elements[:3], indent=2))
except SDKError as e:
    print(e)

INFO: Partitioning without split.
INFO: Successfully partitioned the document.


[
  {
    "type": "Title",
    "element_id": "83da61e09f002098dcee18eb1390b4d7",
    "text": "Policies in Parallel? A Comparative Study of Journalistic AI Policies in 52 Global News Organisations",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "filename": "2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "4f38107d3a45bc16b74fc066546f95c2",
    "text": "PRE-PRINT \u2013 NOT PEER-REVIEWED",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "parent_id": "83da61e09f002098dcee18eb1390b4d7",
      "filename": "2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "0b8cdb6f432263f145c3e61a907891f0",
    "text": "Kim Bj\u00f6rn Becker1\u2020, Felix M. Simon2\u2020, and Christopher Crum3\u2021",
    "metadata": {
 

In [28]:
print(json.dumps(resp.elements[:20], indent=2))

[
  {
    "type": "Title",
    "element_id": "83da61e09f002098dcee18eb1390b4d7",
    "text": "Policies in Parallel? A Comparative Study of Journalistic AI Policies in 52 Global News Organisations",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "filename": "2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "4f38107d3a45bc16b74fc066546f95c2",
    "text": "PRE-PRINT \u2013 NOT PEER-REVIEWED",
    "metadata": {
      "filetype": "application/pdf",
      "languages": [
        "eng"
      ],
      "page_number": 1,
      "parent_id": "83da61e09f002098dcee18eb1390b4d7",
      "filename": "2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf"
    }
  },
  {
    "type": "NarrativeText",
    "element_id": "0b8cdb6f432263f145c3e61a907891f0",
    "text": "Kim Bj\u00f6rn Becker1\u2020, Felix M. Simon2\u2020, and Christopher Crum3\u2021",
    "metadata": {
 

In [27]:
# Print all title elements
[x for x in resp.elements if x['type'] == 'Title']

[{'type': 'Title',
  'element_id': '83da61e09f002098dcee18eb1390b4d7',
  'text': 'Policies in Parallel? A Comparative Study of Journalistic AI Policies in 52 Global News Organisations',
  'metadata': {'filetype': 'application/pdf',
   'languages': ['eng'],
   'page_number': 1,
   'filename': '2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf'}},
 {'type': 'Title',
  'element_id': '2f72eb141c3fba5ace84cc03e7d42b45',
  'text': 'Abstract',
  'metadata': {'filetype': 'application/pdf',
   'languages': ['eng'],
   'page_number': 1,
   'filename': '2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf'}},
 {'type': 'Title',
  'element_id': '9c7e78d47528e0a485f412e38f682429',
  'text': 'Introduction',
  'metadata': {'filetype': 'application/pdf',
   'languages': ['eng'],
   'page_number': 2,
   'filename': '2023_Becker_Simon_Crum_AI_Guidelines_Preprint.pdf'}},
 {'type': 'Title',
  'element_id': '1d7185e377772510be90d2c3fac09039',
  'text': 'Literature Review',
  'metadata': {'filetype': 'applicat

### PDF to Markdown - PyMuPDF

In [9]:
# PyMuPDF
import pymupdf4llm
md_text = pymupdf4llm.to_markdown(filename)

# write markdown string to some file
output = open("out-markdown.md", "w", encoding="utf-8")
output.write(md_text)
output.close()

In [10]:
output

<_io.TextIOWrapper name='out-markdown.md' mode='w' encoding='utf-8'>

# BERTopic

In [60]:
# Read pdf file from folder and extract text from it
# Save the text in a list

import os
import PyPDF2

def read_pdf(file_path):
    pdf_file = open(file_path, 'rb')
    reader = PyPDF2.PdfReader(pdf_file)
    pdf_text = ""
    for page_num in range(len(reader.pages)):
        page = reader.pages[page_num]
        pdf_text += page.extract_text()
    pdf_file.close()
    return pdf_text

folder_path = "Data/"

all_files = []
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):  # Ensure the file is a PDF
        # Construct the full path to the PDF file
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        
        # Read the PDF file
        pdf_text = read_pdf(file_path)
        
        # Append the text to the list
        all_files.append(pdf_text)

Data/2401.05377.pdf
Data/aain-generative-ai-guidelines.pdf
Data/Acceptable uses of generative AI services at IU.pdf
Data/AdC Generative AI Policy.pdf
Data/ai_and_elections_best_practices_no_embargo.pdf
Data/Artificial_Intelligence_Policy_2023.pdf
Data/ChatGPT-Guidance-for-Instructorsc.pdf
Data/ChatGPT_Policy_Brief-v2b.pdf
Data/CIS-Generative-AI-Guidelines-20230214.pdf
Data/Considerations for Integrating AI Within Teaching and Learning.pdf
Data/GAIA-Final-2023.pdf
Data/Generative AI Policy for Teaching and Learning.pdf
Data/Generative_AI_HAI_Perspectives.pdf
Data/Generative_Artificial_Intelligence_(AI)_Use_Policy.pdf
Data/Guidelines-for-the-Use-of-Generative-AI-in-Teaching.pdf
Data/intro_to_gen_ai_tools.pdf
Data/Issues Posed by Generative AI for Teaching and Learning.pdf
Data/ITS Overview Statement - Generative AI Tool Usage - August 2023.pdf
Data/Public Sharing Version - Usage Guidelines Proposal for AI Generative Tools at CU.pdf
Data/Reconsidering Education Policy in the Era of Genera

In [50]:
# Break documents into individual sentences as input
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = [sent_tokenize(file) for file in all_files]
sentences = [sentence for doc in sentences for sentence in doc]

### Preprocess text

In [2]:
import nltk

In [51]:
# Lemmatize
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]
    
# Remove numbers and "et al"
import regex as re
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'et al', '', text)
    return text

### BERTopic Pipeline

In [4]:
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

In [69]:
# Step 1 - Extract embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# Pre compute embeddings so BERTopic doesn't have to calculate them each time the model is run
embeddings = embedding_model.encode(sentences, show_progress_bar=True)
# Try Gensim embeddings?

# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=5, n_components=5, min_dist=0.0, metric='cosine')

# Step 3 - Cluster reduced embeddings
hdbscan_model = HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

# Step 4 - Tokenize topics
vectorizer_model = CountVectorizer(#tokenizer=LemmaTokenizer(),  # custom tokenizer to lemmatize words
                                   #preprocessor=preprocess_text, # custom preprocessor to remove numbers
                                   stop_words="english",    # remove stop words
                                   ngram_range=(1, 3),         # use n-grams
                                   min_df = 5
                                   )                # remove infrequent words

# Step 5 - Create topic representation
ctfidf_model = ClassTfidfTransformer()

# Step 6 - (Optional) Fine-tune topic representations with 
# a `bertopic.representation` model
# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR to diversify topic representation
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
client = openai.OpenAI(api_key=openai_api_key)
prompt = """
I have a topic that contains the following documents: 
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
topic: <topic label>
"""
#openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

Batches: 100%|██████████| 12/12 [00:17<00:00,  1.47s/it]


In [53]:
# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    #"OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    "POS": pos_model
}

In [70]:
# Run the model
topic_model = BERTopic(
  #nr_topics='auto',                         # Automatically determine the number of topics
  embedding_model=embedding_model,          # Step 1 - Extract embeddings
  umap_model=umap_model,                    # Step 2 - Reduce dimensionality
  hdbscan_model=hdbscan_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic represenations

  # Hyperparameters
  #top_n_words=5,
  min_topic_size=5,
  verbose=True,
  calculate_probabilities=True # Calculate the topic probabilities per document
)

# Train model
topics, probs = topic_model.fit_transform(sentences, embeddings)

# Show topics
topic_model.get_topic_info()

2024-06-12 13:00:00,568 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-06-12 13:00:05,032 - BERTopic - Dimensionality - Completed ✓
2024-06-12 13:00:05,036 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-06-12 13:00:05,092 - BERTopic - Cluster - Completed ✓
2024-06-12 13:00:05,097 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-06-12 13:00:23,380 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,28,-1_trust_data_checklist_legal,"[trust, data, checklist, legal, information, m...","[compliance, regulations, policies, policy, se...","[trust, data, checklist, legal, information, m...","[trust, data, checklist, legal, information, m...",[A\nsystem\nshould\nbe\nestablished\nto\nregul...
1,0,64,0_employees_information_company_use,"[employees, information, company, use, data, e...","[employees use, compliance, employees, regulat...","[employees, information, company, use, data, e...","[employees, information, company, use, data, l...",[The\ncontact\ninformation\nfor\nthese\nperson...
2,1,48,1_data_ai systems_generative ai systems_systems,"[data, ai systems, generative ai systems, syst...","[use generative ai, generative ai, generative ...","[data, ai systems, generative ai systems, syst...","[data, systems, deployers, personal, developer...",[The 45th Global Privacy Assembly emphasises ...
3,2,33,2_output_employees_accuracy_systems,"[output, employees, accuracy, systems, generat...","[output, employees use, employee, employees, u...","[output, employees, accuracy, systems, generat...","[output, employees, accuracy, systems, generat...",[>Not circumvent or bypass this or other\ncomp...
4,3,29,3_generative ai tools_tools_ai tools_employees,"[generative ai tools, tools, ai tools, employe...","[use generative ai, generative ai tools, gener...","[generative ai tools, tools, ai tools, employe...","[tools, employees, use, work, generative, cont...",[If\nthere\nare\nspeciﬁc\ngenerative\nAI\ntool...
5,4,29,4_policies_policy_2023_checklist,"[policies, policy, 2023, checklist, use, inter...","[policies, use generative ai, ai, policy, ai t...","[policies, policy, 2023, checklist, use, inter...","[policies, policy, checklist, use, internal, g...",[Use\nin\nCompliance\nwith\nExisting\nLaws\nan...
6,5,28,5_using_content_trust_like,"[using, content, trust, like, generative ai, g...","[use generative ai, using generative ai, gener...","[using, content, trust, like, generative ai, g...","[content, trust, generative, tools, use, new, ...","[Across the country, educators have expressed ..."
7,6,20,6_generative_ai_generative ai_tools,"[generative, ai, generative ai, tools, text, g...","[generative ai tools, use generative ai, gener...","[generative, ai, generative ai, tools, text, g...","[generative, tools, text, content, example, co...",[Generative\nAI\ntools\ncan\ndraft\nemails\nor...
8,7,18,7_accuracy_create_use generative_use generativ...,"[accuracy, create, use generative, use generat...","[use generative ai, generative ai tools, gener...","[accuracy, create, use generative, use generat...","[accuracy, tools, generative, documents, issue...",[• Do not use generative AI images as final cr...
9,8,16,8_bias_tool_ask_work,"[bias, tool, ask, work, questions, organizatio...","[ai tools, bias, ai, agencies, accuracy, tools...","[bias, tool, ask, work, questions, organizatio...","[bias, tool, work, questions, organization, da...",[• What bias is the team introducing in its qu...


In [71]:
topic_model.get_topics()[1]

[('data', 0.09635929832752622),
 ('ai systems', 0.07588574331531964),
 ('generative ai systems', 0.07302081301656224),
 ('systems', 0.06736025635856628),
 ('deployers', 0.06349988867009707),
 ('personal', 0.062487717501879925),
 ('developers', 0.0546494923024435),
 ('generative ai', 0.05363550672810698),
 ('generative', 0.05329922192069416),
 ('ai', 0.04956016526430634)]

### Topic Distribution

In [65]:
docs = [sent_tokenize(file) for file in all_files]
docs

[['1 The impact of generative artificial intelligence on socioeconomic inequalities and \npolicy making  \n  \nValerio Capraro1,*, Austin Lentsch2, Daron Acemoglu3, Selin Akgun4, Aisel Akhmedova4, \nEnnio Bilancini5, Jean -François Bonnefon6, Pablo Brañas -Garza7, Luigi Butera8, Karen M. \nDouglas9, Jim A.C. Everett10, Gerd Gigerenzer11, Christine Greenhow12, Daniel A. \nHashimoto13,14, Julianne Holt -Lunstad15, Jolanda Jetten16, Simon Johnson17, Chiara \nLongoni18, Pete Lunn19, Simone Natale20, Iyad Rahwan21, Neil Selwy n22, Vivek Singh13, \nSiddharth Suri23, Jennifer Sutcliffe4, Joe Tomlinson24, Sander van der Linden25, Paul A. M. \nVan Lange26, Friederike Wall27, Jay J.',
  'Van Bavel28, 29, Riccardo Viale30 \n1 Department of Psychology, University of Milan -Bicocca, Italy.',
  '2 Department of Economics, MIT, USA.',
  '3 \nInstitute Professor and Department of Economics, MIT, USA.',
  '4 College of Education, Michigan State \nUniversity.',
  '5 IMT School of Advanced Studies Lucca,

In [66]:
# Simple topic modeling
#topic_model = BERTopic().fit(all_files)
# Approximate the topic distribution on a token level
topic_distr, _ = topic_model.approximate_distribution(all_files,
                                                      window = 4, # The window size to consider for each token
                                                      use_embedding_model=True) # Use the embedding model to calculate the topic distribution)

topic_model.visualize_distribution(topic_distr[1])

  0%|          | 0/1 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [45]:
# How each token contributes to a specific topic
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(all_files, calculate_tokens=True)

# Visualize the token-level distributions of document indexed 2
topic_model.visualize_approximate_distribution(all_files[3], topic_token_distr[3])


100%|██████████| 1/1 [00:02<00:00,  2.40s/it]


,24,11,14,AM,Acceptable,uses,of,generative,AI,services,at,IU,https,kb,iu,edu,biit,4Acceptable,uses,of,generative,AI,services,at,IU,On,this,age,Overview,Precautions,Acceptable,us,es,Learn,mor,and,et,help,Overview,Note,Micr,osoft,has,begun,ebranding,Bing,Chat,nterpris,as,Micr,osoft,Copilot,During,this,tr,ansition,you,ma,see,either,name,This,is,dis,tinct,fr,om,the,Micr,osoft,36,Copilot,ervice,which,curr,ently,has,limit,ed,vailabilit,at,IU,Sinc,2020,ther,has,been,an,xponential,incr,ease,in,the,in,vestment,and,de,velopment,gener,ative,artiﬁcial,intellig,ence,AI,ervices,Gener,ative,AI,is,ype,artiﬁcial,int,ellig,ence,system,that,ener,ates,ne,text,imag,es,or,other,media,in,espons,to,pr,ompt,Not,able,ener,ative,AI,systems,include,ChatGP,Micr,osoft,Copilot,and,Google,Bard,Precautions,Gener,ative,AI,has,pot,ential,applic,ations,acr,oss,wide,ange,of,indus,tries,including,art,writing,and,oftware,development,However,ther,are,als,concerns,about,the,pot,ential,misus,of,the,se,tools,and,an,dat,shar,ed,with,the,services,When,ou,pr,ovide,inf,ormation,the,se,tools,such,as,querie,student,ssays,grant,pr,opo,sals,source,code,or,datasets,it,is,the,ame,as,po,sting,the,inf,ormation,on,public,ebsite,Indiana,Univ,ersity,enc,ourages,its,aﬃliat,es,to,experiment,with,using,the,se,gener,ative,AI,services,as,long,as,no,institutional,dat,is,submitt,ed,them,without,appr,oval,At,IU,Micr,osoft,Copilot,http,learn,micr,osoft,com,en,us,copilot,overview,formerly,Bing,Chat,nterpris,is,available,or,us,by,IU,acult,and,taﬀ,and,it,is,the,ecommended,ay,to,us,gener,ative,AI,within,the,IU,en,vironment,As


### Topic-Term Matrix

In [47]:
topic_term_matrix = topic_model.c_tf_idf_
topic_term_matrix

<5x112546 sparse matrix of type '<class 'numpy.float64'>'
	with 121554 stored elements in Compressed Sparse Row format>

### Document-Topic Matrix

In [67]:
# Check shape of probs
print(probs.shape)

# Print probs in a dataframe form with sentences indexed and topics as columns
import pandas as pd
probs_df = pd.DataFrame(probs)
probs_df

(373, 3)


,0,1,2
0,0.983279,8.210511e-03,8.510561e-03
1,0.978363,1.063087e-02,1.100574e-02
2,1.000000,1.052679e-309,1.095173e-309
3,1.000000,1.034233e-309,1.073077e-309
4,1.000000,1.034832e-309,1.074487e-309
...,...,...,...
368,1.000000,7.587338e-310,7.650698e-310
369,0.955900,2.205563e-02,2.204418e-02
370,1.000000,8.592394e-310,8.717663e-310
371,1.000000,8.099745e-310,8.168146e-310


### Visualize Topics in 2D

In [72]:
# Check if the topic model has been fitted
if hasattr(topic_model, "topic_embeddings_") and topic_model.topic_embeddings_ is not None:
    print("Topic embeddings shape:", topic_model.topic_embeddings_.shape)
    if topic_model.topic_embeddings_.shape[0] == 0:
        raise ValueError("Topic embeddings are empty. Ensure the model has been fitted with data.")
else:
    raise ValueError("Topic model has not been fitted or topic embeddings are not available.")

# Proceed with visualization if embeddings are valid
topic_model.visualize_topics()


Topic embeddings shape: (16, 384)


In [73]:
# Topic similarity
topic_model.visualize_heatmap()

In [74]:
# Visualize keywords per topic 
topic_model.visualize_barchart()

In [75]:
# Visualize documents
topic_model.visualize_documents(sentences, embeddings=embeddings)

In [90]:
import matplotlib

# Approximate topic distribution
# Calculate the topic distributions on a token-level
topic_distr, topic_token_distr = topic_model.approximate_distribution(sentences, calculate_tokens=True)

# Visualize the token-level distributions
topic_model.visualize_approximate_distribution(sentences[1], topic_token_distr[1])


100%|██████████| 1/1 [00:00<00:00,  4.14it/s]


ImportError: background_gradient requires matplotlib.

## Gensim LDA

In [ ]:
!pip install scipy==1.8.0
!pip install gensim==4.3.2

In [59]:
!pip uninstall scipy


^C


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
from gensim import corpora
from nltk.tokenize import word_tokenize
import nltk

documents = [
    "Human machine interface for Lab ABC computer applications",
    "A survey of user opinion of computer system response time",
    "The EPS user interface management system",
    "System and human system engineering testing of EPS"
]

texts = [word_tokenize(document.lower()) for document in documents]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

from gensim.models.ldamodel import LdaModel

lda = LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)
topics = lda.print_topics(num_words=4)
for topic in topics:
    print(topic)

# RAG

In [ ]:
import re
import os
import sqlite3

from langchain_community.document_loaders import PyPDFLoader

folder_path = "Data/Non-profits and NGOs/"

# Initialize a list to store the pages from all PDFs
all_pages = []

# Iterate through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):  # Ensure the file is a PDF
        # Construct the full path to the PDF file
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        
        # Initialize the PyPDFLoader for the current PDF file
        loader = PyPDFLoader(file_path)
        
        # Load and split the pages of the current PDF file
        pages = loader.load()
        print(pages)
        
        # Extend the list of all pages with the pages from the current PDF
        all_pages.extend(pages)

## Split

In [38]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

# Split the text of all pages
all_splits = text_splitter.split_documents(all_pages)

In [42]:
len(all_splits)
len(all_splits[0].page_content)
all_splits[0]
all_splits[20].metadata

{'source': 'Data/Non-profits and NGOs/5.-Resolution-on-Generative-AI-Systems-101023.pdf',
 'page': 4,
 'start_index': 2}

## Embed and Store

In [35]:
import os
import getpass


In [44]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

embedding = OpenAIEmbeddings(model="text-embedding-3-small",
                             api_key=openai_api_key
                             )
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embedding)

## Retrieval

In [53]:
# Set up retriever with vector store
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("What obligations do GenAI developers have?")
print(retrieved_docs[0].page_content)

control, transparency and  democratic values;   
 
Developers, providers  and deployers of generative AI systems should embed data protection and 
privacy in the conception, design, operation and management of new products and services using 
generative AI systems, based on the principles of data protection and privacy by design and 
document their choices and analyses in a data protection and privacy impact assessment;  
 
Moreover, developers, providers and deployers of generative AI systems should understand the 
risks, harms, and potential impact on affected individuals and society  at large as the basis to 
develop ethical, trustworthy and responsible generative AI;  
 
In addition, developers, providers and deployers of generative AI systems should also put in place 
measures to ensure compliance with data protection and privacy obligat ions. These stakeholders 
should cooperate to ensure that individuals whose data are processed by generative AI systems


## Generation

In [46]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 api_key=openai_api_key)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [54]:
# Customize the prompt
from langchain_core.prompts import PromptTemplate
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt #default_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What obligations do GenAI developers have?")

'GenAI developers have obligations to embed data protection and privacy in their systems, understand the risks and potential impact on individuals and society, and ensure compliance with data protection and privacy obligations. They must also avoid developing or using AI systems that could lead to unfair, unethical, or discriminatory treatment. Thanks for asking!'